# Demo RNA Multi-Perceptrón Backpropagation para identificar clase de IRIS
Adaptado de https://www.tensorflow.org/tutorials/estimator/premade

1) Cargar librerías:

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd

from  sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("Librerías cargadas")

2) Cargar los datos:

In [ ]:
# define atributos y clases
CSV_COLUMN_NAMES = ['LargoSepalo', 'AnchoSepalo', 'LargoPetalo', 'AnchoPetalo', 'Clase']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# levanta los datos de entrenamiento y prueba
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

# genera los datos solo com la clase para entrenar y probar
train_y = train.pop('Clase')
test_y = test.pop('Clase')

print("\n\nDatos Originales ", len(train)+len(test))
print("- Datos para Entrenar ", len(train))
print("- Datos para Probar ", len(test))

In [ ]:
# ver datos
train.head()
#test.head()

3) Definir funciones auxiliares para el entrenamiento:

In [ ]:
# funciones auxiliares para entrenar y probar 
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

4) Establecer el modelo para la RNA

In [ ]:
# Construcye un estimador tipo RNA 'Deep' Neuronal Network classifier
classifier = tf.estimator.DNNClassifier(
    
    feature_columns=my_feature_columns,

    hidden_units=[20, 8], # 2 capas de 20 y 8 neuronas ocultas    
    
    n_classes=3) # identifica 3 clases


5) Entrenar el modelo de la RNA:

In [ ]:
# Train the Model.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=10000) 

6) Evaluar el modelo de la RNA entrenado:

In [ ]:
# evalúa el modelo con los datos de test
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nExactitud: {accuracy:0.3f}\n'.format(**eval_result))

7) Mostrar el detalle con predicciones sobre datos de Prueba:

In [ ]:
# funciones auxiliares
def input_predict_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# ejecuta el modelo
predictions = classifier.predict(
    input_fn=lambda: input_predict_fn(test))

# muestra resultados de la predicción
classPreds = []
classReal = []
cantOK = 0
cantError = 0
print("\n Resultados: ")
for pred_dict, expec in zip(predictions, test_y):
    pred_class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][pred_class_id]
    
    classPreds.append(SPECIES[pred_class_id])
    classReal.append(SPECIES[expec])

    if (pred_class_id == expec): 
      res = ""
      cantOK += 1
    else: 
      res = "!"
      cantError += 1

    print('Clase predecida es "{}"[{}] ({:.1f}%), la correcta es "{}"[{}]{} '.format(
        SPECIES[pred_class_id], pred_class_id, 100 * probability, SPECIES[expec], expec, res))

print('=== Total Ejemplos: {}, con {} predicciones ok y {} errores.'.format(cantOK+cantError, cantOK, cantError))

# gráfico de comparación
plt.title('Gráfico de Confusión')
plt.xlabel('Real')
plt.ylabel('RNA')
plt.scatter(classReal, classPreds)

# muestra reporte de clasificación
print("\n Reporte de Clasificación: ")
print(classification_report(classReal, classPreds))

# muestra matriz de confusion
print('\nMatriz de Confusión: ')
cm = confusion_matrix(classReal, classPreds, labels=SPECIES)
cmtx = pd.DataFrame(
    cm, 
    index=['r:{:}'.format(x) for x in SPECIES], 
    columns=['p:{:}'.format(x) for x in SPECIES]
  )
print(cmtx)
print("\n")


*) Hacer pruebas de predicciones:

In [ ]:
# define ejemplos de prueba
expected = ['Setosa', 'Versicolor', 'Virginica']

predict_x = {
    'LargoSepalo': [5.1, 5.9, 6.9],
    'AnchoSepalo': [3.3, 3.0, 3.1],
    'LargoPetalo': [1.7, 4.2, 5.4],
    'AnchoPetalo': [0.5, 1.5, 2.1],
}

In [ ]:
# funciones auxiliares
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

# muestra resultados de la predicción
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Clase predecida es "{}" ({:.1f}%), la correcta es "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))